In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [3]:
# 1. Load Data
# Replace 'your_dataset.csv' with your actual file name
df = pd.read_csv('/Users/ravikumarnalawade/Documents/Certifications/Project/GitHub/Credit-Risk-Analysis/data/lcDataSample.csv', low_memory=False)

print(f"Initial Shape: {df.shape}")

Initial Shape: (110000, 145)


In [4]:
# 2. Define the Target: Loan Status
# We only care about completed loans: 'Fully Paid' vs 'Charged Off'
target_mask = df['loan_status'].isin(['Fully Paid', 'Charged Off', 'Default'])
df_clean = df[target_mask].copy()

# Encode Target: 1 for Bad Loan (Default/Charged Off), 0 for Good
df_clean['target'] = df_clean['loan_status'].apply(lambda x: 1 if x in ['Charged Off', 'Default'] else 0)

print(f"Filtered Shape: {df_clean.shape}")
print("Target Distribution:\n", df_clean['target'].value_counts(normalize=True))

Filtered Shape: (109944, 146)
Target Distribution:
 target
0    0.860138
1    0.139862
Name: proportion, dtype: float64


The Trap: If you build a lazy model that just guesses "Good Loan" (0) for everyone, you will get 86% Accuracy.

The Reality: That model is useless. It catches 0% of the fraudsters/defaulters, which is the whole point of the business.


When I analyzed the target variable, I found a significant class imbalance—only 14% of loans were defaults. This meant Accuracy was a misleading metric. I shifted my optimization focus to Recall (capturing as many defaulters as possible) and ROC-AUC to measure the model's true discriminatory power.

In [5]:
# 3. CRITICAL: Remove Data Leakage
# These columns reveal the future. We must drop them to simulate real-world prediction.
leakage_cols = [
    'recoveries', 'collection_recovery_fee', 'total_pymnt', 'total_pymnt_inv',
    'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'last_pymnt_d',
    'last_pymnt_amnt', 'next_pymnt_d', 'debt_settlement_flag'
]

# Drop leakage + original target text column
cols_to_drop = leakage_cols + ['loan_status', 'id', 'member_id', 'url', 'desc']
# Only drop if they exist in dataset
existing_drops = [c for c in cols_to_drop if c in df_clean.columns]
df_clean = df_clean.drop(columns=existing_drops)

In [6]:
# 4. Feature Engineering: Date Handling
# Convert date strings to datetime objects
date_cols = ['issue_d', 'earliest_cr_line']
for col in date_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Feature: Credit History Length (in years)
# We calculate the difference between the Loan Issue Date and Earliest Credit Line
if 'issue_d' in df_clean.columns and 'earliest_cr_line' in df_clean.columns:
    # Remove timezone info from issue_d to match earliest_cr_line
    df_clean['issue_d'] = df_clean['issue_d'].dt.tz_localize(None)
    df_clean['credit_hist_years'] = (df_clean['issue_d'] - df_clean['earliest_cr_line']).dt.days / 365
    print("Feature Engineered: 'credit_hist_years'")

Feature Engineered: 'credit_hist_years'


/var/folders/1y/fk3nmszx1jv_9_cy47zq59dc0000gn/T/ipykernel_74535/2725885748.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')


In [7]:
# 5. Quick EDA Check
# Let's look at correlation of numeric features with the target
numeric_df = df_clean.select_dtypes(include=[np.number])
correlations = numeric_df.corr()['target'].sort_values(ascending=False)

print("\nTop 5 Positively Correlated Features (Higher value = Higher Risk):")
print(correlations.head(6)) # Top 1 is target itself

print("\nTop 5 Negatively Correlated Features (Higher value = Lower Risk):")
print(correlations.tail(5))


Top 5 Positively Correlated Features (Higher value = Higher Risk):
target                  1.000000
dti_joint               0.228373
int_rate                0.189151
inq_last_12m            0.128271
open_rv_24m             0.111657
acc_open_past_24mths    0.098334
Name: target, dtype: float64

Top 5 Negatively Correlated Features (Higher value = Lower Risk):
debt_settlement_flag_date   NaN
settlement_status           NaN
settlement_date             NaN
settlement_amount           NaN
settlement_percentage       NaN
Name: target, dtype: float64


In [8]:
import os

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Export df_clean as pickle (preserves dtypes, fastest)
df_clean.to_pickle('../data/df_clean.pkl')

# Also export as CSV for easy inspection
df_clean.to_csv('../data/df_clean.csv', index=False)

# Export column names to text file
with open('../data/column_names.txt', 'w') as f:
    f.write('\n'.join(df_clean.columns))

print(f"✓ df_clean exported successfully!")
print(f"  - Pickle: ../data/df_clean.pkl ({df_clean.shape[0]} rows, {df_clean.shape[1]} cols)")
print(f"  - CSV: ../data/df_clean.csv")
print(f"  - Column names: ../data/column_names.txt")

✓ df_clean exported successfully!
  - Pickle: ../data/df_clean.pkl (109944 rows, 131 cols)
  - CSV: ../data/df_clean.csv
  - Column names: ../data/column_names.txt
